# Getting Started with ctapipe

This hands-on was presented at the Paris CTA Consoritum meeting (K. Kosack)

## Part 1: load and loop over data

In [ ]:
from ctapipe.io import event_source
from ctapipe import utils
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
path = utils.get_dataset_path("gamma_test_large.simtel.gz")

In [ ]:
source = event_source(path, max_events=4)

for event in source:
    print(event.count, event.index.event_id, event.mc.energy)

In [ ]:
event

In [ ]:
event.r0

In [ ]:
for event in event_source(path, max_events=4):
    print(event.count, event.r0.tels_with_data)

In [ ]:
event.r0.tel[2]

In [ ]:
r0tel = event.r0.tel[2]

In [ ]:
r0tel.waveform

In [ ]:
r0tel.waveform.shape

note that this is ($N_{channels}$, $N_{pixels}$, $N_{samples}$)

In [ ]:
plt.pcolormesh(r0tel.waveform[0])

In [ ]:
plt.plot(r0tel.waveform[0,10])

In [ ]:
from ipywidgets import interact

@interact
def view_waveform(chan=0, pix_id=200):
    plt.plot(r0tel.waveform[chan, pix_id])

try making this compare 2 waveforms

## Part 2: Explore the instrument description
This is all well and good, but we don't really know what camera or telescope this is... how do we get instrumental description info?

Currently this is returned *inside* the event (it will soon change to be separate in next version or so)

In [ ]:
subarray = source.subarray 


In [ ]:
subarray

In [ ]:
subarray.peek()

In [ ]:
subarray.to_table()

In [ ]:
subarray.tel[2]

In [ ]:
subarray.tel[2].camera

In [ ]:
subarray.tel[2].optics

In [ ]:
tel = subarray.tel[2]

In [ ]:
tel.camera

In [ ]:
tel.optics

In [ ]:
tel.camera.geometry.pix_x

In [ ]:
tel.camera.geometry.to_table()

In [ ]:
tel.optics.mirror_area

In [ ]:
from ctapipe.visualization import CameraDisplay

In [ ]:
disp = CameraDisplay(tel.camera.geometry)

In [ ]:
disp = CameraDisplay(tel.camera.geometry)
disp.image = r0tel.waveform[0,:,10]  # display channel 0, sample 0 (try others like 10)

 ** aside: ** show demo using a CameraDisplay in interactive mode in ipython rather than notebook

## Part 3: Apply some calibration and trace integration

In [ ]:
from ctapipe.calib import CameraCalibrator

In [ ]:
calib = CameraCalibrator(subarray=subarray)

In [ ]:
for event in event_source(path, max_events=4):
    calib(event) # fills in r1, dl0, and dl1
    print(event.dl1.tel.keys())

In [ ]:
event.dl1.tel[2]

In [ ]:
dl1tel = event.dl1.tel[2]

In [ ]:
dl1tel.image.shape # note this will be gain-selected in next version, so will be just 1D array of 1855

In [ ]:
dl1tel.peak_time

In [ ]:
CameraDisplay(tel.camera.geometry, image=dl1tel.image)

In [ ]:
CameraDisplay(tel.camera.geometry, image=dl1tel.peak_time)

Now for Hillas Parameters

In [ ]:
from ctapipe.image import hillas_parameters, tailcuts_clean

In [ ]:
image = dl1tel.image
mask = tailcuts_clean(tel.camera.geometry, image, picture_thresh=10, boundary_thresh=5)
mask

In [ ]:
CameraDisplay(tel.camera.geometry, image=mask)

In [ ]:
cleaned = image.copy()
cleaned[~mask] = 0 

In [ ]:
disp = CameraDisplay(tel.camera.geometry, image=cleaned)
disp.cmap = plt.cm.coolwarm
disp.add_colorbar()
plt.xlim(-1.0,0)
plt.ylim(0,1.0)

In [ ]:
params = hillas_parameters(tel.camera.geometry, cleaned)
print(params)

In [ ]:
disp = CameraDisplay(tel.camera.geometry, image=cleaned)
disp.cmap = plt.cm.coolwarm
disp.add_colorbar()
plt.xlim(-1.0,0)
plt.ylim(0,1.0)
disp.overlay_moments(params, color='white', lw=2)

## Part 4:  Let's put it all together: 
- loop over events, selecting only telescopes of the same type (e.g. LST:LSTCam)
- for each event, apply calibration/trace integration
- calculate Hillas parameters 
- write out all hillas paremeters to a file that can be loaded with Pandas

first let's select only those telescopes with LST:LSTCam

In [ ]:
subarray.telescope_types

In [ ]:
subarray.get_tel_ids_for_type("LST_LST_LSTCam")

Now let's write out program

In [ ]:
data = utils.get_dataset_path("gamma_test_large.simtel.gz") 
source = event_source(data, allowed_tels=[1,2,3,4],  max_events=10) # remove the max_events limit to get more stats

In [ ]:
for event in source:
    calib(event)
    
    for tel_id, tel_data in event.dl1.tel.items():
        tel = source.subarray.tel[tel_id]
        mask = tailcuts_clean(tel.camera.geometry, tel_data.image)
        params = hillas_parameters(tel.camera.geometry[mask], tel_data.image[mask])

In [ ]:
from ctapipe.io import HDF5TableWriter


In [ ]:
with HDF5TableWriter(filename='hillas.h5', group_name='dl1', overwrite=True) as writer:
    
    source = event_source(data, allowed_tels=[1,2,3,4],  max_events=10)
    for event in source:
        calib(event)
    
        for tel_id, tel_data in event.dl1.tel.items():
            tel = source.subarray.tel[tel_id]
            mask = tailcuts_clean(tel.camera.geometry, tel_data.image)
            params = hillas_parameters(tel.camera.geometry[mask], tel_data.image[mask])
            writer.write("hillas", params)

### We can now load in the file we created and plot it

In [ ]:
!ls *.h5

In [ ]:
import pandas as pd

hillas = pd.read_hdf("hillas.h5", key='/dl1/hillas')
hillas

In [ ]:
_ = hillas.hist(figsize=(8,8))

If you do this yourself, loop over more events to get better statistics